In [1]:
from scipy.io import loadmat
data = loadmat('trainingData.mat')
data = data['dataRecord']


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
torch.backends.quantized.engine = 'qnnpack'
# 假设你的数据是numpy数组形式，需要转换成torch Tensor
import numpy as np

# 示例数据，你应该用你的实际数据替换这里的np.random.rand()
inputs = torch.tensor(data[:, :6], dtype=torch.float32)
outputs = torch.tensor(data[:, 6:], dtype=torch.float32)

# 创建数据集和数据加载器
dataset = TensorDataset(inputs, outputs)
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)

# 定义神经网络模型
class SimpleFNN(nn.Module):
    def __init__(self):
        super(SimpleFNN, self).__init__()
        self.fc1 = nn.Linear(6, 6)  # 输入层到隐藏层，50个神经元
        self.fc2 = nn.Linear(6, 3)  # 输出层

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        return self.fc2(x)

model = SimpleFNN()
model.train()

# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 训练网络
epochs = 200
for epoch in range(epochs):
    for inputs, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')


Epoch 1, Loss: 0.018325302749872208
Epoch 2, Loss: 0.004156238865107298
Epoch 3, Loss: 0.0005332004511728883
Epoch 4, Loss: 5.9412992413854226e-05
Epoch 5, Loss: 1.460906150896335e-05
Epoch 6, Loss: 1.2239222996868193e-05
Epoch 7, Loss: 2.053016760328319e-05
Epoch 8, Loss: 1.1803905181295704e-05
Epoch 9, Loss: 8.64634148456389e-06
Epoch 10, Loss: 7.730958714091685e-06
Epoch 11, Loss: 1.591099498909898e-05
Epoch 12, Loss: 1.4091124285187107e-05
Epoch 13, Loss: 1.1602066479099449e-05
Epoch 14, Loss: 1.3366987332119606e-05
Epoch 15, Loss: 1.3509876225725748e-05
Epoch 16, Loss: 1.0530505278438795e-05
Epoch 17, Loss: 1.0298735105607193e-05
Epoch 18, Loss: 5.448203864943935e-06
Epoch 19, Loss: 1.1805386748164892e-05
Epoch 20, Loss: 1.6013578715501353e-05
Epoch 21, Loss: 1.1528681170602795e-05
Epoch 22, Loss: 1.3763986316916998e-05
Epoch 23, Loss: 9.433938430447597e-06
Epoch 24, Loss: 1.625661570869852e-05
Epoch 25, Loss: 1.1688028280332219e-05
Epoch 26, Loss: 7.980598638823722e-06
Epoch 27, 

In [8]:
# infer and save
test_input = torch.tensor([[0.11412254, -0.09698317, -0.13648682, -0.08168529,  0.09584415,
        -0.08081015]], dtype=torch.float32)  # 示例输入数据
with torch.no_grad():
    prediction = model(test_input)
    print(f'Prediction: {prediction.numpy()}')
    print("-0.01443261, -0.01519005, -0.02687549")
# save the model
scripted_model = torch.jit.script(model)
torch.jit.save(scripted_model, '../model/model.pth')

Prediction: [[-0.01431365 -0.01424116 -0.02677222]]
-0.01443261, -0.01519005, -0.02687549


In [9]:
for name, param in model.named_parameters():
    # 保存权重
    np.savetxt(f"{name}_weight.txt", param.data.cpu().numpy())

    # 检查是否存在偏置，并保存
    if hasattr(param, 'bias') and param.bias is not None:
        np.savetxt(f"{name}_bias.txt", param.bias.data.cpu().numpy())

In [4]:
model.named_parameters()

<generator object Module.named_parameters at 0x321720d40>

In [71]:
data = loadmat('testingData.mat')
test_data = data['dataRecord']

# 测试数据集，应该与训练数据集具有相同的结构
test_inputs = torch.tensor(test_data[:, :6], dtype=torch.float32)
test_outputs = torch.tensor(test_data[:, 6:], dtype=torch.float32)

# 创建测试 DataLoader
test_dataset = TensorDataset(test_inputs, test_outputs)
test_loader = DataLoader(test_dataset, batch_size=10)

# 将模型设置为评估模式
model.eval()

# 初始化损失计算
test_loss = 0.0
with torch.no_grad():  # 确保不进行梯度计算
    for inputs, labels in test_loader:
        # 前向传播
        outputs = model(inputs)
        # 计算损失
        loss = criterion(outputs, labels)
        test_loss += loss.item()

# 平均损失
test_loss /= len(test_loader)
print(f'Average loss on the test dataset: {test_loss}')


Average loss on the test dataset: 5.30677266397106e-07


In [72]:
qm = torch.quantization.quantize_dynamic(model, {nn.Linear,torch.relu}, dtype=torch.qint8)
qm.eval()

# 初始化损失计算
test_loss = 0.0
with torch.no_grad():  # 确保不进行梯度计算
    for inputs, labels in test_loader:
        # 前向传播
        outputs = qm(inputs)
        # 计算损失
        loss = criterion(outputs, labels)
        test_loss += loss.item()

# 平均损失
test_loss /= len(test_loader)
print(f'Average loss on the test dataset: {test_loss}')
scripted_qmodel = torch.jit.script(qm)
torch.jit.save(scripted_qmodel, '../model/qmodel.pth')

Average loss on the test dataset: 4.0735253321599884e-07
